# Assignment 2: Convolutional Autoencoders

Britt Schmitz  - i6235053 <br>
Tabea Heusel - i6323791

## 0 Setup

### 0.1 Imports

In [20]:
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split

### 0.2 Loading Dataset

In [18]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
X = np.append(x_train, x_test, axis=0)
y = np.append(y_train, y_test, axis=0)

## 1 Autoencoder Network Implementation

### 1.1 Train, Validation, and Test 

In [24]:
X_train, X_remaining, y_train, y_remaining = train_test_split(X, y, test_size=0.2, random_state=3)
X_val, X_test, y_val, y_test = train_test_split(X_remaining, y_remaining, test_size=0.5, random_state=3)

### 1.2 Autoencoder Network